In [30]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date

In [4]:
#equipa de limpeza teste
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1")
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1")
df_tags = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1")

In [8]:
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

In [20]:
#limpeza
#ownerid passar para int
#data tirar 00
#title string
#body string
#year int
#df_questions = df_questions.dropna()
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')

In [51]:
#df_questions.dtypes
# df_questions.head()
# print(df_questions['CreationDate']
df_questions['day'] = df_questions['CreationDate'].dt.day
df_questions['month'] = df_questions['CreationDate'].dt.month
df_questions['year'] = df_questions['CreationDate'].dt.year
df_questions['hour'] = df_questions['CreationDate'].dt.hour
df_questions['minute'] = df_questions['CreationDate'].dt.minute
df_questions['second'] = df_questions['CreationDate'].dt.second
df_questions.head()

# frames = [df_questions['day'], df_questions['month'], df_questions['year'], df_questions['hour'], df_questions['minute'], df_questions['second']]
# result = pd.concat(frames, axis=1)
# result

df_questions['Date1'] = df_questions[df_questions.columns[6:9]].apply(
     lambda x: '/'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['Date2'] = df_questions[df_questions.columns[9:12]].apply(
     lambda x: ':'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['Date3'] = df_questions[df_questions.columns[13:15]].apply(
     lambda x: ' '.join(x.dropna().astype(str)),
     axis=1
)
#df_questions.head()

In [52]:
#criacao tabelas mongo
df_questions.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,day,month,year,hour,minute,second,Date,Date1,Date2,Date3
0,469,147,2008-08-02 15:11:16+00:00,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,2,8,2008,15,11,16,"2,8,2008,15,11",2/8/2008,15:11:16,2/8/2008 15:11:16
1,502,147,2008-08-02 17:01:58+00:00,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,2,8,2008,17,1,58,"2,8,2008,17,1",2/8/2008,17:1:58,2/8/2008 17:1:58
2,535,154,2008-08-02 18:43:54+00:00,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,2,8,2008,18,43,54,"2,8,2008,18,43",2/8/2008,18:43:54,2/8/2008 18:43:54
3,594,116,2008-08-03 01:15:08+00:00,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,3,8,2008,1,15,8,"3,8,2008,1,15",3/8/2008,1:15:8,3/8/2008 1:15:8
4,683,199,2008-08-03 13:19:16+00:00,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,3,8,2008,13,19,16,"3,8,2008,13,19",3/8/2008,13:19:16,3/8/2008 13:19:16


In [ ]:
#criacao tabelas mysql

In [ ]:
queries = [
    {"description": "Retrieve all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2008", "query": ""},
    
    {"description": "Retrive all questions that have 2 tags", "query": ""},

    {"description": "Retrive all questions from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [ ]:
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'score' : {'$gt' : 300}}
        result = collection_stackoverflow.find(query)

    elif "answers from 2008" in q["description"]:
        query = {'creationdate': {'$regex' : '^2008-.*'}}
        result = collection_stackoverflow.find(query)

    elif "2 tags" in q["description"]:
        query = {}
        result = 
    else:
        query = {}
        result = 
        
    end_time = time.time()
 
    list_result = list(result)
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {len(list_result)} found in {end_time - start_time:.4f} seconds")
client.close()

In [ ]:
mysql_times = []
with engine.connect() as connection:
    for q in queries:
        start_time = time.time()
        if "score greater" in q["description"]:
            result = connection.execute(text("SELECT title FROM questions WHERE score > 300"))

        elif "answers from 2008" in q["description"]:
            result = connection.execute(text("SELECT * FROM answers WHERE creationdate LIKE '2008%'"))
        
        elif "2 tags" in q["description"]:
            result = connection.execute(text("(SELECT id FROM tags GROUP BY id HAVING COUNT(tag) = 2) AS t JOIN questions q ON t.id = q.id"))
            
        else:
            result = connection.execute(text("SELECT title FROM questions WHERE creationdate LIKE '2015%' AND title LIKE '%PDF%' AND (SELECT COUNT(*) FROM answers WHERE parentid = questions.id) > 4")) #maybe we need a join to compare parentid to questions.id
        results =  result.fetchall()
        end_time = time.time()
        mysql_times.append(end_time - start_time)
        print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)